# Instant LLM Term Explanations with Live LLM
### NON-LLM Detection & Color-Coded Answers.

As an LLM student, I wanted a lightweight tool to speed up learning: one that explains technical terms concisely, flags whether a question falls under LLM topics, and points out when a term is not actually LLM-related. The interface color-codes answers (green for LLM topics, orange otherwise) and provides short notes so you can quickly confirm or correct your assumptions.

In [21]:
# imports

import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import HTML

# Load env (allow values in .env to override existing env if you want)
load_dotenv(override=True)

True

In [22]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2' # Direct ollama cloud models work out of the box e.g. 'gpt-oss:120b-cloud'

# Pull the model
!ollama list | grep -q "$MODEL_LLAMA" && echo "✅ $MODEL_LLAMA already exists" || (echo "🚀 Pulling $MODEL_LLAMA model..." && echo "⏳ This may take a few minutes depending on size..." && ollama pull "$MODEL_LLAMA" && echo "✅ Download completed! $MODEL_LLAMA is ready.")


✅ llama3.2 already exists


In [23]:
# environment

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")

In [24]:
# clients

openai_client = OpenAI(api_key=OPENAI_API_KEY)
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY)

In [25]:
# here is the question; type over this to ask something new

system_prompt = (
    "You are an assistant whose job is to (1) decide whether the user's question is about "
    "large language models (LLMs) or any topic within the LLM domain (architecture, training, "
    "fine-tuning, distillation, quantization, prompting, prompt engineering, tokenization, "
    "embeddings, inference, evaluation, safety, deployment, retrieval, adapters, LoRA, etc.), "
    "and (2) answer the question. Use your judgment — do not output your internal chain-of-thought. "
    "Return ONLY a single valid JSON object with these fields:\n"
    '  { "is_llm_related": true|false, "color": "green"|"orange", '
    '"answer": "<MANDATORY!!! Your answer should not be more than 100 words>", "note": "<MANADAOTORY!!! Write a tiny note when not LLM-related>" }\n'
    "If the question is LLM-related, set is_llm_related=true and color='green'. "
    "If not, set is_llm_related=false and color='orange' and include a short note. "
    "Do not include any extra text before or after the JSON."
)

def create_prompt(user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ]


In [26]:
def display_response(content):
    c = json.loads(content)
    html = f"Answer: <p style='color:{c['color']}'>{c['answer']}</p>"
    if not c['is_llm_related']:
        html += f"<br><small>Out of scope: </small><small style='color:gray'>{c['note']}</small>"
    return display(HTML(html))


### OpenAI `gpt-4o-mini` Model

- ##### Example of a prompt within the LLM domain

In [27]:
good_response = openai_client.chat.completions.create(
    model=MODEL_GPT,
    messages=create_prompt("What is amnesia in large language models?")
)
display_response(good_response.choices[0].message.content)

- ##### Example of a prompt outside the LLM domain

In [28]:
bad_response = openai_client.chat.completions.create(
    model=MODEL_GPT,
    messages=create_prompt("Who built the Eiffel Tower?")
)
display_response(bad_response.choices[0].message.content)

### Ollama `llama3.2` Model

- ##### Example of a prompt within the LLM domain

In [29]:
good_response = ollama_client.chat.completions.create(
    model=MODEL_LLAMA,
    messages=create_prompt("What is hydration in large language models?")
)
display_response(good_response.choices[0].message.content)

- ##### Example of a prompt outside the LLM domain

In [30]:
bad_response = ollama_client.chat.completions.create(
    model=MODEL_LLAMA,
    messages=create_prompt("Who is the president of the United States?")
)
display_response(bad_response.choices[0].message.content)